In [1]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import base64

In [2]:
from jina.flow import Flow
import os 

os.environ['PARALLEL'] = str(4)
os.environ['SHARDS'] = str(2)
os.environ['DATA_DIR'] = 'data/jpg'
os.environ['COLOR_CHANNEL_AXIS'] = str(0)
os.environ['JINA_PORT'] = str(45678)
os.environ['ENCODER'] = 'yaml/encode.yml'
os.environ['WORKDIR'] = 'data/workspace'
os.environ['TMP_WORKSPACE'] = 'data/workspace'
f = Flow.load_config('./flow-query.yml')

           JINA@19962[W]:parser Flow can not recognize the following args: ['--read-only', '--port-expose', '45678'], they are ignored. if you are using them from a global args (e.g. Flow), then please ignore this message
           Flow@19962[S]:successfully built Flow from a yaml config


In [3]:
f.use_grpc_gateway()

In [4]:
result_html = []

def print_result(resp):
    for d in resp.search.docs:
        vi = d.uri
        result_html.append(f'<tr><td><img src="{vi}"/></td><td>')
        for match in d.matches:
            im = Image.open(io.BytesIO(match.meta_info))
            fname="{}.jpg".format(match.id)
            fname = os.path.join(os.environ['TMP_RESULTS'], fname)
            im.save(fname)
            result_html.append(f'<img src="{fname}" />')
        result_html.append('</td></tr>\n')


def write_html(html_path):
    with open(resource_filename('jina', '/'.join(('resources', 'helloworld.html'))), 'r') as fp, \
            open(html_path, 'w') as fw:
        t = fp.read()
        t = t.replace('{% RESULT %}', '\n'.join(result_html))
        fw.write(t)

    url_html_path = 'file://' + os.path.abspath(html_path)

    try:
        webbrowser.open(url_html_path, new=2)
    except:
        pass
    

In [ ]:
image_src = './image_*.jpg'
with f:
    f.search_files(image_src, sampling_rate=.01, batch_size=8, output_fn=print_result, top_k=5)

         loader@19981[I]:post initiating, this may take some time...
         loader@19981[I]:post initiating, this may take some time takes 0.002 secs
         loader@19981[S]:successfully built ImageReader from a yaml config
         loader@19981[I]:setting up sockets...
         loader@19981[I]:input tcp://0.0.0.0:45675 (PULL_BIND) 	 output tcp://0.0.0.0:38185 (PUSH_CONNECT)	 control over tcp://0.0.0.0:40219 (PAIR_BIND)
         loader@19981[S]:ready and listening
        flipper@19996[I]:post initiating, this may take some time...
        flipper@19996[I]:post initiating, this may take some time takes 0.002 secs
        flipper@19996[S]:successfully built ImageFlipper from a yaml config
        flipper@19996[I]:setting up sockets...
        flipper@19996[I]:input tcp://0.0.0.0:38185 (PULL_BIND) 	 output tcp://0.0.0.0:40737 (PUSH_CONNECT)	 control over tcp://0.0.0.0:49263 (PAIR_BIND)
        flipper@19996[S]:ready and listening
     normalizer@20011[I]:post initiating, this may take

  chunk_indexer@20101[I]:indexer size: 501
  chunk_indexer@20101[C]:unknown exception: PermissionError(13, 'Permission denied')
Traceback (most recent call last):
  File "/home/s/miniconda3/envs/jina/lib/python3.7/site-packages/jina/peapods/pea.py", line 332, in run
    self.loop_body()
  File "/home/s/miniconda3/envs/jina/lib/python3.7/site-packages/jina/peapods/pea.py", line 309, in loop_body
    self.load_executor()
  File "/home/s/miniconda3/envs/jina/lib/python3.7/site-packages/jina/peapods/pea.py", line 193, in load_executor
    self.args.separated_workspace, self.args.replica_id)
  File "/home/s/miniconda3/envs/jina/lib/python3.7/site-packages/jina/executors/__init__.py", line 416, in load_config
    return yaml.load(tmp_s)
  File "/home/s/miniconda3/envs/jina/lib/python3.7/site-packages/ruamel/yaml/main.py", line 343, in load
    return constructor.get_single_data()
  File "/home/s/miniconda3/envs/jina/lib/python3.7/site-packages/ruamel/yaml/constructor.py", line 113, in get_si

In [ ]:
write_html('result.html')

In [ ]:
pip install torchvision